In [1]:
import os
import glob
import json
import pandas as pd

In [2]:
!cp -r /kaggle/input/dict-v2/dict /kaggle/working/dict

In [3]:
root = '/kaggle/input'
des_path = '/kaggle/working/dict'

# Part 1

In [4]:
data_root_list = ['/kaggle/input/aic2024-keyframes0102/Keyframes', '/kaggle/input/aic2024-keyframes0304/Keyframes', '/kaggle/input/aic2024-keyframes0506/Keyframes', '/kaggle/input/aic2024-keyframes0708/Keyframes',  '/kaggle/input/aic2024-keyframes0910/Keyframes', '/kaggle/input/aic2024-keyframes1112/Keyframes', '/kaggle/input/aic-keyframe-1314/Keyframes', '/kaggle/input/aic-keyframe-1516/Keyframes', '/kaggle/input/aic-keyframe-1718/Keyframes', '/kaggle/input/aic-keyframe-1920/Keyframes', '/kaggle/input/aic-keyframe-2122/Keyframes', '/kaggle/input/aic-keyframe-2324/Keyframes']
metadata_root = root + '/dict-v2/dict/metadata'
scene_root = root + '/dict-v2/dict/SceneJSON'
map_keyframes_root = root + '/dict-v2/dict/map_keyframes'
fps_path = root + '/dict-v2/dict/fps.json'

with open(fps_path, 'r') as f:
    fps_dict = json.load(f)

In [5]:
scene_id2info = dict()

for data_part in sorted(os.listdir(scene_root)):
    data_part_path = f'{scene_root}/{data_part}'
    scene_id2info[data_part] = dict()
    for video_path in sorted(os.listdir(data_part_path)):
        video_id = video_path.split('.')[0]
        video_path_full = f'{data_part_path}/{video_path}'
        video_fps = fps_dict[f'{data_part}_{video_id}']
        
        with open(video_path_full, 'r') as f:
            video_scene_info = json.load(f)
            
        with open(f'{metadata_root}/{data_part}_{video_id}.json', 'r', encoding="utf8") as f:
            video_metadata = json.load(f)
        
        scene_id2info[data_part][video_id] = {
            'video_metadata': video_metadata,
            'lst_shot': dict()
        }
        
        for i, item in enumerate(video_scene_info):
            scene_id2info[data_part][video_id]['lst_shot'][str(i)] = {
                'shot_range': item,
                'shot_time': [item[0]/video_fps, item[1]/video_fps],
                'lst_keyframe_paths': [],
                'lst_keyframe_idxs': []
            }

In [6]:
global_index = 0
id2img_fps = []

for data_root in data_root_list:
    for data_part in sorted(os.listdir(data_root)):
        data_part_path = f'{data_root}/{data_part}'

        for video_id in sorted(os.listdir(data_part_path)):
            video_dir = f'{data_part_path}/{video_id}'

            temp_data_part = data_part.replace('_extra', '') # replace L01_extra -> L01
            with open(f'{scene_root}/{temp_data_part}/{video_id}.json', 'r') as f:
                video_scene_info = json.load(f)

            if 'extra' not in data_part:
                mapper = pd.read_csv(f'{map_keyframes_root}/{data_part}_{video_id}.csv', index_col='n')

            scene_track = 0
            for image_path in sorted(os.listdir(video_dir)):
                frame_idx = int(image_path.split('.')[0])

                if 'extra' not in data_part:
                    frame_idx = int(mapper.loc[frame_idx]['frame_idx'])

                image_path = f'{video_dir}/{image_path}'.replace(f'{data_root}', '/data/Keyframes')
    
                while(len(video_scene_info) > 0 and frame_idx > video_scene_info[0][1]):
                    video_scene_info.pop(0)
                    scene_track += 1

                if len(video_scene_info) == 0:
                    continue

                info = {
                    "image_path": image_path,
                    "scene_idx": f'{temp_data_part}/{video_id}/lst_shot/{str(scene_track)}'
                }

                scene_id2info[temp_data_part][video_id]['lst_shot'][str(scene_track)]['lst_keyframe_paths'].append(image_path)
                scene_id2info[temp_data_part][video_id]['lst_shot'][str(scene_track)]['lst_keyframe_idxs'].append(global_index)

                id2img_fps.append(info)

                global_index += 1

In [7]:
id2img_fps = dict(enumerate(id2img_fps))

In [8]:
with open(f'{des_path}/scene_id2info.json', 'w') as f:
    f.write(json.dumps(scene_id2info))
    
with open(f'{des_path}/id2img_fps.json', 'w') as f:
    f.write(json.dumps(id2img_fps))

print(f'Saved {des_path}')
print(f"Number of Index: {len(id2img_fps)}")

Saved /kaggle/working/dict
Number of Index: 856121


In [9]:
global_index

856121

# Part 2

In [10]:
map_keyframes_dict = dict()
for video_path in sorted(os.listdir(map_keyframes_root)):
    data_part, video_id = video_path.replace('.csv', '').split('_')
    mapper = pd.read_csv(f'{map_keyframes_root}/{data_part}_{video_id}.csv', index_col='n')
    
    key = f'{data_part}_{video_id}'
    map_keyframes_dict[key] = dict() 
    
    for index in mapper.index.values:
        map_keyframes_dict[key][int(index)] = int(mapper.loc[index]['frame_idx'])

In [11]:
with open(f'{des_path}/map_keyframes.json', 'w') as f:
    f.write(json.dumps(map_keyframes_dict))

# Part 3

In [12]:
with open(f'{des_path}/scene_id2info.json', 'r') as f:
    SceneID2Info = json.load(f)

In [13]:
audios_detection_dir = f'/kaggle/input/dict-v2/dict/audio_detection'

check_error = 0
audio_id2img_fps = []
for data_part in sorted(os.listdir(audios_detection_dir)):
    for audio_detection_path in sorted(os.listdir(f'{audios_detection_dir}/{data_part}')):
        audio_id = audio_detection_path.replace('.json', '')
        scene_info = SceneID2Info[data_part][audio_id]['lst_shot']
        
        with open(f'{audios_detection_dir}/{data_part}/{audio_detection_path}', 'r') as f:
            audio_shots = json.load(f)
            
        i = 0
        scene_info_len = len(scene_info)
        for audio_interval in audio_shots:
            result = []
            start, end = audio_interval
                
            while True:
                if i >= scene_info_len:
                    break
                
                shot_interval = scene_info[str(i)]['shot_time']
                if end <= shot_interval[0]:
                    break
                if(start >= shot_interval[1]):
                    i += 1
                    continue
                    
                result.extend(scene_info[str(i)]['lst_keyframe_idxs'].copy()) 
                if end > shot_interval[1]:
                    i += 1
                    start = shot_interval[1]
                else:
                    break
                    
            audio_id2img_fps.append(result)
            
            
            check_error += 1

In [14]:
if len(audio_id2img_fps) != check_error:
    print("Some thing not good!!")
else:
    print("Successfully")
    
audio_id2img_fps = dict(enumerate(audio_id2img_fps))

Successfully


In [15]:
with open(f'{des_path}/audio_id2img_id.json', 'w') as f:
    f.write(json.dumps(audio_id2img_fps))

# Part 5

In [16]:
with open(f'{des_path}/scene_id2info.json', 'r') as f:
    SceneID2Info = json.load(f)

In [17]:
video_id2img_id = dict()

In [18]:
for data_part in SceneID2Info.keys():
    for video_id in SceneID2Info[data_part].keys():
        sample_key = f'{data_part}_{video_id}'
        video_id2img_id[sample_key] = []
        for key, value in SceneID2Info[data_part][video_id]['lst_shot'].items():
            video_id2img_id[sample_key].extend(value['lst_keyframe_idxs'])

In [19]:
with open(f'{des_path}/video_id2img_id.json', 'w') as f:
    f.write(json.dumps(video_id2img_id))

# Part 6

In [20]:
with open(f'{des_path}/scene_id2info.json', 'r') as f:
    SceneID2Info = json.load(f)

In [21]:
import numpy as np
from tqdm import tqdm

In [22]:
def find_nearest(array, value):     
    array = np.asarray(array)
    idx = sorted((np.abs(array - value)).argsort()[:2].tolist())
    return idx

In [23]:
audios_detection_dir = f'/kaggle/input/dict-v2/dict/audio_detection'

audio_global_id = 0
img_id2audio_id = dict()
for data_part in tqdm(sorted(os.listdir(audios_detection_dir))):
    for audio_detection_path in sorted(os.listdir(f'{audios_detection_dir}/{data_part}')):
        audio_id = audio_detection_path.replace('.json', '')
        scene_info = SceneID2Info[data_part][audio_id]['lst_shot']
        
        with open(f'{audios_detection_dir}/{data_part}/{audio_detection_path}', 'r') as f:
            audio_shots = json.load(f)
        
        audio_pivot_shots = []
        for audio_shot in audio_shots:
            start, end = audio_shot
            audio_pivot_shots.append((start+end)/2)
        
        for shot in scene_info.values():
            shot_center = (shot['shot_time'][0] + shot['shot_time'][1])/2
            shot_frame_idxs = shot['lst_keyframe_idxs']
            nearest_audio = [audio_global_id + val for val in find_nearest(audio_pivot_shots, shot_center)]
            
            for shot_frame_idx in shot_frame_idxs:
                img_id2audio_id[shot_frame_idx] = nearest_audio
            
        audio_global_id += len(audio_shots)

100%|██████████| 24/24 [00:05<00:00,  4.63it/s]


In [24]:
with open(f'{des_path}/img_id2audio_id.json', 'w') as f:
    f.write(json.dumps(img_id2audio_id))